In [ ]:
import json
import redis
from langchain_ollama import OllamaLLM
from langchain.document_loaders import PyPDFLoader
from langchain_ollama import OllamaEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# Connect to Redis
redis_client = redis.StrictRedis(host='localhost', port=6379, db=0, decode_responses=True)

# Store data in Redis
def store_in_redis(key, data):
    redis_client.set(key, json.dumps(data))

# Retrieve data from Redis
def retrieve_from_redis(key):
    data = redis_client.get(key)
    if data:
        return json.loads(data)
    return None

conversation_history_key = 'conversation_history'
vectorstore_key = 'vectorstore'

def load_pdf_and_create_vector_store(pdf_path):
    """Loads a PDF file, creates embeddings, and stores them in FAISS."""
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()

    if not docs:
        raise ValueError("🚨 No documents were loaded from the PDF! Check if the file is valid.")

    embeddings = OllamaEmbeddings(model="deepseek-llm:7b-chat")
    vectorstore = FAISS.from_documents(docs, embeddings)
    
    # Store vector store in Redis
    store_vector_store_in_redis(vectorstore, vectorstore_key)
    return vectorstore

def store_vector_store_in_redis(vectorstore, key):
    """Serialize and store vector store in Redis."""
    vectorstore_data = vectorstore.save()  # Assuming FAISS has a `save()` method
    store_in_redis(key, vectorstore_data)

def retrieve_vector_store_from_redis(key):
    """Retrieve the vector store from Redis."""
    vectorstore_data = retrieve_from_redis(key)
    if vectorstore_data:
        return FAISS.load(vectorstore_data)  # Assuming FAISS has a `load()` method
    return None

def query_agent(question: str, vectorstore, llm):
    """Queries the vectorstore using the LLM and retrieves the answer."""
    try:
        # Retrieve documents relevant to the current query
        retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
        retrieved_docs = retriever.invoke(question)

        if not retrieved_docs:
            return {
                "answer": "I don't know.",
                "explanation": "No relevant information found in the documents.",
                "source_documents": []
            }

        context = "\n".join([doc.page_content for doc in retrieved_docs if doc.page_content.strip()])

        # Retrieve conversation history from Redis
        global conversation_history
        conversation_history = retrieve_from_redis(conversation_history_key) or []

        if conversation_history:
            # Join previous conversation history into the context
            conversation_context = "\n".join([f"Q: {entry['question']}\nA: {entry['answer']}" for entry in conversation_history])
            context = conversation_context + "\n" + context

        QA_PROMPT = PromptTemplate(
            template="""Use the following context to answer the question. If the context doesn't contain the answer, say 'I don't know'.

            Context: {context}
            Question: {query}

            Provide the answer in JSON format:
            {{
                "answer": "your concise answer here",
                "explanation": "your detailed explanation here"
            }}
            """,
            input_variables=["context", "query"],
        )

        formatted_prompt = QA_PROMPT.format(context=context, query=question)

        response = llm.invoke(formatted_prompt).strip()

        try:
            parsed_response = json.loads(response)
        except json.JSONDecodeError:
            parsed_response = {
                "answer": response,
                "explanation": "The model did not return a structured JSON response."
            }

        # Save the current question and answer to conversation history
        conversation_history.append({
            "question": question,
            "answer": parsed_response.get("answer", "No answer found"),
        })
        
        # Store updated conversation history in Redis
        store_in_redis(conversation_history_key, conversation_history)

        return {
            "answer": parsed_response.get("answer", "No answer found"),
            "explanation": parsed_response.get("explanation", "No explanation provided"),
            "source_documents": [doc.page_content for doc in retrieved_docs if doc.page_content.strip()],
            "context": conversation_history
        }

    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return {
            "answer": "Error occurred while processing the query",
            "explanation": str(e),
            "source_documents": []
        }

if __name__ == "__main__":
    llm = OllamaLLM(model="deepseek-llm:7b-chat")

    # Load vectorstore from Redis or from PDF
    vectorstore = retrieve_vector_store_from_redis(vectorstore_key)
    if not vectorstore:
        vectorstore = load_pdf_and_create_vector_store("English.pdf")

    query = "please convert you answer into markdown and add headings to the last answer"
    result = query_agent(query, vectorstore, llm)
    print(json.dumps(result, indent=2))
